<a href="https://colab.research.google.com/github/Graziottomp/ImageFileConvertion/blob/main/RecorteImagem_LeafMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
!/usr/bin/env python
coding: utf-8
Rotina para recortar uma imagem GeoTiff manualmente com visualização pela biblioteca LeafMap:
author = 'Maria Paula Graziotto' adaptado de 'Qiusheng WuVer'
copyright = "Copyright 2023, Maria Paula Graziotto"
credits = ['Graziotto, M. P.']
license = "Public Domain"
version = "1.0"
maintainer = "Maria Paula Graziotto"
email = 'graziotto.mp@outlook.com'
-------------------------------------------------------------------------------
```



In [ ]:
!pip install segment-geospatial leafmap
!pip install localtileserver
!pip install rasterio xarray rioxarray

In [2]:
import os
import leafmap
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff
import rasterio as rs
import xarray as xr
import rioxarray as rio
import pandas as pd
from rasterio.windows import Window

In [10]:
#Definindo diretório e arquivo (recomendo fazer um arquivo por vez, sem looping)
current_dir = os.path.dirname

raster_file = "file_name.tiff"

In [11]:
# Carregando a imagem e seus atributos
with rs.open(raster_file) as src:
#Transformando o src do raster
    raster_data = src.read()
    transform = src.transform

# Definindo as bordas do raster
    xmin, ymin, xmax, ymax = src.bounds

# Calculando o centro do raster
    center_lat = (ymin + ymax) / 2
    center_lon = (xmin + xmax) / 2

In [ ]:
# Mostrando a imagem georeferenciada e interativa utilizando LeafMap
m = leafmap.Map(center=[center_lat, center_lon], zoom=9)
m.add_basemap("SATELLITE")
m.add_layer_control()
m.layers[-1].visible = False
m.add_raster(raster_file, layer_name="Image")
m

In [14]:
#Opção de usar o bbox ou a área selecionada no mapa
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = src.bounds

In [15]:
#Criar e salvar a imagem recortada com os atributos
#nome do arquivo e o diretório
output_raster_path = "file_name.tif"

#Bbox pré-determinado
xmin, ymin, xmax, ymax = [bbox[0], bbox[1], bbox[2], bbox[3]]

#criando o novo raster
with rs.open(raster_file) as src2:
    row_start, col_start = src2.index(xmin, ymax)
    row_stop, col_stop = src2.index(xmax, ymin)

#Recorte do subset
    window = Window.from_slices((row_start, row_stop), (col_start, col_stop))

#Lendo o subset do novo raster
    subset_data = src2.read(window=window)

# Adicionando o metadata no novo raster
    subset_profile = src2.profile.copy()
    subset_profile.update({
        'height': window.height,
        'width': window.width,
        'transform': src2.window_transform(window)
    })

#Armazenando o metadata gerado
    with rs.open(output_raster_path, 'w', **subset_profile) as dst:
        dst.write(subset_data)